# Zadanie A

In [1]:
def read_fasta(filename) -> list:
    f = open(filename)

    sequence = ""
    sequences = []

    for line in f:
        if line.startswith(">"):
            if len(sequence) != 0:
                sequences.append(sequence)
                sequence = ""
        else:
            sequence += line.strip()

    if sequence:
        sequences.append(sequence)

    return sequences


In [2]:
def write_fasta(filename, sequences):
    f = open(filename, 'w')
    for sequence in sequences:
        f.write('>\n')

        for idx in range(0, len(sequence), 70):
            f.write(sequence[idx:idx+70])
            f.write('\n')

In [3]:
import random

def generate_reads(sequence_file):
    genom = read_fasta(sequence_file)[0]

    read_length = 200
    coverage = 5
    gen_len = len(genom)

    all_reads = gen_len - 1 - read_length
    one_read_prob = read_length / gen_len
    reads_num = 1
    while one_read_prob * reads_num < coverage:
        reads_num += 1

    reads = []
    for _ in range(reads_num):
        start = random.randint(0, gen_len - read_length)
        reads.append(genom[start:start+read_length])

    write_fasta('sequence_reads.fasta', reads)


# Zadanie B

In [4]:
def join(first : str, second :str) -> str:
    joined = None
    for length in range(1, max(len(first), len(second))):
        try:
            if first.endswith(second[:length]):
                joined = first + second[length:]
            elif second.endswith(first[:length]):
                joined = second + first[length:]
        except:
            pass

    return joined

In [5]:
import itertools

class Cache:
    def __init__(self, strings):
        self._cache = dict()

        for first, second in itertools.product(strings, repeat=2):
            if first == second:
                continue
        
            joined = join(first, second)
            
            if len(frozenset([first, second])) == 1:
                print(frozenset([first, second]))

            if joined:
                self._cache[frozenset([first, second])] = joined

    def remove(self, string):
        for key in list(self._cache):
            if string in key:
                del self._cache[key]

    def add(self, string, strings):
        for other in strings:
            if string == other:
                continue

            joined = join(string, other)

            if not joined:
                continue
            
            if string == joined or other == joined:
                continue
            
            self._cache[frozenset([string, other])] = joined

    def get(self, first, second) -> str:
        key = frozenset([first, second])

        if key in self._cache:
            return self._cache[key]
        else:
            return None

    def get_max(self):
        if self._cache:
            return max(self._cache.keys(), key=lambda x: len(self._cache[x]))
        else:
            return None, None
    
    def __len__(self):
        return len(self._cache)

In [6]:
def build_contigs():
    reads_set = set(read_fasta('sequence_reads.fasta'))
    sequence_pairs = Cache(reads_set)

    import itertools
    from collections import namedtuple
    Contig = namedtuple('Contig', ['first', 'second', 'contig'])

    while len(reads_set) != 1:
        if len(sequence_pairs) == 0: # there is no sequence fragmets to create new contigs
            break

        maxContig = None

        key = list(sequence_pairs.get_max())
        first, second = key[0], key[1]
        joined = sequence_pairs.get(first, second)

        if not joined:
            continue

        if not maxContig:
            maxContig = Contig(first, second, joined)
        elif joined != first and joined != second:
            maxContig = max(maxContig, Contig(first, second, joined), key=lambda x: len(x.contig))

        if maxContig:
            reads_set.discard(maxContig.first)
            reads_set.discard(maxContig.second)
            reads_set.add(maxContig.contig)

            sequence_pairs.remove(maxContig.first)
            sequence_pairs.remove(maxContig.second)
            sequence_pairs.add(maxContig.contig, reads_set)

            print('sequence fragmets left', len(reads_set))
        else:
            break

    write_fasta('sequence_contigs.fasta', reads_set)

# Zadanie C

In [7]:
sequence_file = './human_genome_short_fragment.fasta'
# sequence_file = './human_genome_long_fragment.fasta'

generate_reads(sequence_file)
build_contigs()

sequence fragmets left 83
sequence fragmets left 82
sequence fragmets left 81
sequence fragmets left 80
sequence fragmets left 79
sequence fragmets left 78
sequence fragmets left 77
sequence fragmets left 76
sequence fragmets left 75
sequence fragmets left 74
sequence fragmets left 73
sequence fragmets left 72
sequence fragmets left 71
sequence fragmets left 70
sequence fragmets left 69
sequence fragmets left 68
sequence fragmets left 67
sequence fragmets left 66
sequence fragmets left 65
sequence fragmets left 64
sequence fragmets left 63
sequence fragmets left 62
sequence fragmets left 61
sequence fragmets left 60
sequence fragmets left 59
sequence fragmets left 58
sequence fragmets left 57
sequence fragmets left 56
sequence fragmets left 55
sequence fragmets left 54
sequence fragmets left 53
sequence fragmets left 52
sequence fragmets left 51
sequence fragmets left 50
sequence fragmets left 49
sequence fragmets left 48
sequence fragmets left 47
sequence fragmets left 46
sequence fra